In [6]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
#ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141812
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-141812


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
Cluster = 'ClusterProject1'
Compute_Target = ComputeTarget(workspace = ws, name=Cluster)
Compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_v2', max_nodes=4, min_nodes=1)
Compute_Target = Compute_Target.create(ws, Cluster, Compute_config)


In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# Specify parameter sampler
ps = RandomParameterSampling({"--C" : choice(0.01, 0.1, 1), "--max_iter" : choice(20, 40, 60, 100, 150, 200, 300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', compute_target = Cluster, entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig ( 
    hyperparameter_sampling=ps,
    primary_metric_name='Accruracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    policy=policy,
    estimator=est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c31e5fc1-d4af-441b-9fc3-21d193dbf15c
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c31e5fc1-d4af-441b-9fc3-21d193dbf15c?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-141812/workspaces/quick-starts-ws-141812

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-02T14:20:09.766733][API][INFO]Experiment created<END>\n""<START>[2021-04-02T14:20:10.854100][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2021-04-02T14:20:11.067708][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-02T14:20:11.1492111Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_c31e5fc1-d4af-441b-9fc3-21d193dbf15c
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c31e5fc1-d4af-441b-9fc3-21d193dbf15c?wsid=/subscriptions/b96

{'runId': 'HD_c31e5fc1-d4af-441b-9fc3-21d193dbf15c',
 'target': 'ClusterProject1',
 'status': 'Canceled',
 'startTimeUtc': '2021-04-02T14:20:09.551833Z',
 'endTimeUtc': '2021-04-02T14:40:16.37094Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": 

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel',model_path = '/outputs/model.joblib')

AttributeError: 'NoneType' object has no attribute 'id'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
full_data = pd.concat([x,predict_col],axis=1)
train_data, test_data = train_test_split(full_data,test_size=0.25)
train_data.to_csv("bankmarket_training.csv")

data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='training/')
final_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'training/bankmarket_training.csv')])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=classidication,
    primary_metric='Accuracy',
    training_data=final_data,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl = exp.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = '/outputs/')